<a href="https://colab.research.google.com/github/wirungu/limit-order-book/blob/main/Limit_Order_Book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A simple limit order book
As part of a recent technical interview I was given the following problem, which I enjoyed thinking about. What follows is a cleaned up version of my submission, along with some comments. There may well still be bugs in this code, even though it returns the correct solution for our test case. I'd love to hear about them if you find any.

## Problem statement
Implement a class `LimitOrderBook` that can process limit orders, i.e., either execute them against existing orders in the book or add them to the book. The limit order book should maintain the `best bid`, `best bid volume` (sum of volume at best bid), `best ask` and `best ask volume` (sum of volume at best ask).

Your class should be able to process the following list of orders:



In [4]:
orders = [
    {"direction": "buy", "quantity": 100, "price": 98},
    {"direction": "buy", "quantity": 20, "price": 99},
    {"direction": "buy", "quantity": 50, "price": 99},
    {"direction": "sell", "quantity": 100, "price": 101},
    {"direction": "sell", "quantity": 200, "price": 101},
    {"direction": "sell", "quantity": 200, "price": 102},
    {"direction": "sell", "quantity": 120, "price": 99},
]

After processing these orders, what is the `best_bid`, `best_bid_volume`, `best_ask`, `best_ask_volume`?

## Problem Overview

A limit order is a direction to buy or sell a certain amount of stock at a specified price or better, in contrast to a market order which is fulfilled at the prevailing price. A limit order book is a record of outstanding limit orders maintained by the exchange. A buy limit order is an order to buy at a pre-specified price or lower while a sell limit order is an order to sell a security at a pre-specified price or higher. The bid side of the book then represents the open offers to buy, while the ask side represents open orders to sell. The buyers wish to buy for as low a price as possible, and the sellers wish to sell for as high a price as possible, and there develops a *bid-ask spread.* A trade happens when the highest bid is greater than or equal to the lowest ask, and we say the spread has been crossed.

Limit order books are often implemented with price-time priority: orders are executed at the best possible price first, and if there are multiple outstanding orders at the same price the earliest one is chosen. This is not what's going on in our example here since there aren't any timestamps.

Now, let's think about the process for submitting (e.g.) a buy order:
- First, check the lowest price of the sell side of the limit book.
- If this price is less than or equal to the the buy side, execute a trade.
- If the buyer still has more volume left to fill, look for the next lowest price on the sell side and keep going.
- If there is still unfilled volume for the buyer's trade after doing this, add that volume to the bid side.

We are prioritizing orders with the lowest price, so we're looking for a priority queue implementation. What data structure should we use to store this order book?
A heap is only one possibility: $O(\log n)$ inserts and pops worst case. We'll use a min heap to represent the sell side (and quickly get the lowest price) and a max heap to represent the buy side. Unfortunately python's `heapq` can't directly implement max heaps: one has to negate the prices and use a min heap, which doesn't feel very pythonic aside from the many bugs that surely must arise from such an approach.

If we had timestamps, we would implement a heap of queues (assuming the orders are coming in order of timestamp).

## Implementation

In [3]:
import heapq

class LimitOrderBook:
    def __init__(self):
        self.buy_orders = []  # Max heap for buy orders (negate price to simulate max heap)
        self.sell_orders = []  # Min heap for sell orders
        self.best_bid = None
        self.best_bid_volume = 0
        self.best_ask = None
        self.best_ask_volume = 0

    def process_orders(self, order_list):
        for order in order_list:
            direction = order["direction"]
            quantity = order["quantity"]
            price = order["price"]

            if direction == "buy":
                self.process_buy_order(quantity, price)
            elif direction == "sell":
                self.process_sell_order(quantity, price)

    def process_buy_order(self, quantity, price):
        # Update best bid and best bid volume
        if not self.best_bid or price > self.best_bid:
            self.best_bid = price
            self.best_bid_volume = quantity
        elif price == self.best_bid:
            self.best_bid_volume += quantity

        heapq.heappush(self.buy_orders, (-price, quantity))

    def process_sell_order(self, quantity, price):
        # Update best ask and best ask volume
        if not self.best_ask or price < self.best_ask:
            self.best_ask = price
            self.best_ask_volume = quantity
        elif price == self.best_ask:
            self.best_ask_volume += quantity

        heapq.heappush(self.sell_orders, (price, quantity))

        while self.buy_orders and self.sell_orders:
            best_buy_price = -self.buy_orders[0][0]
            best_sell_price = self.sell_orders[0][0]

            if best_buy_price < best_sell_price:
                break

            buy_price, buy_quantity = heapq.heappop(self.buy_orders)
            sell_price, sell_quantity = heapq.heappop(self.sell_orders)

            trade_quantity = min(buy_quantity, sell_quantity)
            if trade_quantity > 0:
                self.best_bid_volume -= trade_quantity
                self.best_ask_volume -= trade_quantity

                if buy_quantity > trade_quantity:
                    heapq.heappush(self.buy_orders, (buy_price, buy_quantity - trade_quantity))
                if sell_quantity > trade_quantity:
                    heapq.heappush(self.sell_orders, (sell_price, sell_quantity - trade_quantity))

        # Update best bid and best ask if orders were completely filled
        if self.best_bid_volume == 0 and self.buy_orders:
            self.best_bid = -self.buy_orders[0][0]
            self.best_bid_volume = self.buy_orders[0][1]

        if self.best_ask_volume == 0 and self.sell_orders:
            self.best_ask = self.sell_orders[0][0]
            self.best_ask_volume = self.sell_orders[0][1]


Let's run this. The set of orders here is small enough that we can compute the best bid and best ask ourselves to verify the result.

In [5]:
order_book = LimitOrderBook()
order_book.process_orders(orders)

print("Best Bid:", order_book.best_bid)
print("Best Bid Volume:", order_book.best_bid_volume)
print("Best Ask:", order_book.best_ask)
print("Best Ask Volume:", order_book.best_ask_volume)

Best Bid: 98
Best Bid Volume: 100
Best Ask: 99
Best Ask Volume: 50


A simple check by hand shows that this is indeed correct.

## Explanation of code
Here's an explanation of the key components and functionalities of the code:

1. **Class Initialization (`__init__` method):**
   - The constructor initialized two heaps: `buy_orders` and `sell_orders`. The `buy_orders` heap is a max heap (implemented using negated prices), while the `sell_orders` heap is a min heap.
   - The attributes `best_bid`, `best_bid_volume`, `best_ask`, and `best_ask_volume` are initialized to track the best bid and ask prices, as well as their corresponding total volumes.

2. **Processing Orders (`process_orders` method):**
   - The `process_orders` method takes a list of orders as input and processes each order one by one.
   - For each order, the direction (buy or sell), quantity, and price are extracted.

3. **Processing Buy Orders (`process_buy_order` method):**
   - When processing a buy order (`direction` is "buy"), the method updates the best bid and best bid volume if needed.
   - It adds the buy order to the `buy_orders` heap while negating the price (to ensure max heap behavior).

4. **Processing Sell Orders (`process_sell_order` method):**
   - When processing a sell order (`direction` is "sell"), the method updates the best ask and best ask volume if needed.
   - It adds the sell order to the `sell_orders` heap.

5. **Matching Orders (`while` loop):**
   - After processing each order, the code enters a `while` loop to match buy and sell orders if possible.
   - It compares the best buy price and the best sell price. If the best buy price is higher than or equal to the best sell price, a trade can occur.
   - Inside the loop, it pops the best buy and sell orders from their respective heaps.
   - It calculates the trade quantity as the minimum of the buy and sell quantities.
   - The trade quantity is subtracted from the best bid and best ask volumes.
   - If there's remaining quantity for the buy or sell order, the updated order is pushed back onto the respective heap.

6. **Heap Operations:**
   - The use of heaps (`buy_orders` and `sell_orders`) ensures efficient retrieval of the best bid and best ask prices.
   - The negation of prices for buy orders allows for proper max heap behavior.

7. **Best Bid and Best Ask Updates:**
   - The best bid and best ask prices and volumes are updated based on incoming orders and trades.
8. **Check if Best Bid and Best Ask are Filled:**
   - If all orders at the best bid and best ask price level were completely filled, we update to the next best bid/ask price and volume.


## Alternative implementations
Besides using two heaps for the buy and sell orders, there are several other data structures that we could consider for implementing a limit order book, each with their own tradeoffs:

1. Doubly-Linked List
2. AVL Tree or Red-Black Tree
3. Hash Table
4. Skip Lists
5. Fibonacci Heap

People in industry seem to prefer linked lists where the buy and sell orders are separate doubly-linked lists. Each node in the doubly-linked list represents an order, and the nodes would be linked together to maintain the order of arrival.

Balanced BSTs also seem popular in industry.

## Some useful references
1. Several posts on https://quant.stackexchange.com were helpful: just search for limit order books to find them.
2. https://web.archive.org/web/20110219163448/http://howtohft.wordpress.com/2011/02/15/how-to-build-a-fast-limit-order-book/
3. The book *Limit Order Books* by Frédéric Abergel et al.
4. The paper *Limit order books* by Martin D. Gould et al.